In [6]:
import cv2
import mediapipe as mp
import numpy as np
from pygame import mixer
import time
import os

# ---------------- CAMERA ----------------
cap = cv2.VideoCapture(0)

# ---------------- ALARM ----------------
mixer.init()
sound = mixer.Sound(r"C:\Users\HP\Downloads\alarm-301729.mp3")

# ---------------- SAVE ALERT IMAGES ----------------
SAVE_DIR = "alerts"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---------------- MEDIAPIPE ----------------
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# ---------------- LANDMARK INDICES ----------------
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH = [78, 81, 13, 311, 308, 14]

# ---------------- FUNCTIONS ----------------
def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    return (A + B) / (2.0 * C)

def mouth_aspect_ratio(mouth):
    A = np.linalg.norm(mouth[1] - mouth[5])
    B = np.linalg.norm(mouth[2] - mouth[4])
    C = np.linalg.norm(mouth[0] - mouth[3])
    return (A + B) / (2.0 * C)

# ---------------- PARAMETERS ----------------
CALIBRATION_TIME = 5        # seconds
CLOSED_TIME_LIMIT = 5      # seconds 
YAWN_THRESHOLD = 0.6
BLINK_MAX_TIME = 0.25

adaptive_ear_threshold = None
ear_samples = []
calibration_start = time.time()

eyes_closed_start = None
alarm_on = False
screenshot_taken = False

blink_count = 0
blink_start = None
yawn_count = 0
drowsiness_score = 0

# ---------------- MAIN LOOP ----------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Mirror image
    frame = cv2.flip(frame, 1)

    h, w = frame.shape[:2]
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        face = results.multi_face_landmarks[0]

        left_eye = np.array([(int(face.landmark[i].x * w),
                              int(face.landmark[i].y * h)) for i in LEFT_EYE])
        right_eye = np.array([(int(face.landmark[i].x * w),
                               int(face.landmark[i].y * h)) for i in RIGHT_EYE])
        mouth = np.array([(int(face.landmark[i].x * w),
                           int(face.landmark[i].y * h)) for i in MOUTH])

        ear = (eye_aspect_ratio(left_eye) + eye_aspect_ratio(right_eye)) / 2
        mar = mouth_aspect_ratio(mouth)

        # ---------------- CALIBRATION ----------------
        if adaptive_ear_threshold is None:
            ear_samples.append(ear)
            cv2.putText(frame, "Calibrating... Keep eyes open",
                        (30,40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,0), 2)

            if time.time() - calibration_start >= CALIBRATION_TIME:
                adaptive_ear_threshold = np.mean(ear_samples) * 0.75
            cv2.imshow("Driver Drowsiness System", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        # ---------------- BLINK DETECTION ----------------
        if ear < adaptive_ear_threshold:
            if blink_start is None:
                blink_start = time.time()
        else:
            if blink_start:
                if time.time() - blink_start <= BLINK_MAX_TIME:
                    blink_count += 1
                blink_start = None

        # ---------------- YAWN DETECTION ----------------
        if mar > YAWN_THRESHOLD:
            yawn_count += 1
            time.sleep(0.6)

        # ---------------- DROWSINESS (10-SECOND RULE) ----------------
        if ear < adaptive_ear_threshold:
            if eyes_closed_start is None:
                eyes_closed_start = time.time()

            closed_time = time.time() - eyes_closed_start
            drowsiness_score += 1

            cv2.putText(frame, f"Eyes Closed: {int(closed_time)}s",
                        (30,70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)

            if closed_time >= CLOSED_TIME_LIMIT and not alarm_on:
                sound.play()
                alarm_on = True

                if not screenshot_taken:
                    cv2.imwrite(f"{SAVE_DIR}/alert_{int(time.time())}.jpg", frame)
                    screenshot_taken = True

                print("ALERT SENT TO MOBILE / IOT SYSTEM")

        else:
            eyes_closed_start = None
            screenshot_taken = False
            drowsiness_score = max(0, drowsiness_score - 2)

            if alarm_on:
                sound.stop()
                alarm_on = False

        # ---------------- DISPLAY (BLACK COLOR) ----------------
        cv2.putText(frame, f"Blinks: {blink_count}", (30,40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)

        cv2.putText(frame, f"Yawns: {yawn_count}", (30,100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)

        cv2.putText(frame, f"Drowsiness Score: {drowsiness_score}",
                    (30,130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)

    cv2.imshow("Driver Drowsiness Monitoring System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🚨 ALERT SENT TO MOBILE / IOT SYSTEM
